In [1]:
def load_data():
    """
    Load product data from a 'dati_prodotti.txt' file and return a list of dictionaries containing product information.
    """
    try:
        with open('dati_prodotti.txt', 'r') as file:
            products = []
            for line in file:
                product = line.strip().split(',')
                products.append({
                    'nome': product[0],
                    'quantita': int(product[1]),
                    'prezzo_acquisto': float(product[2]),
                    'prezzo_vendita': float(product[3])
                })
            return products
    except FileNotFoundError:
        return []
    except ValueError as e:
        print(f"Errore durante il caricamento dei dati: {e}")
        return []

In [2]:
def save_data(products):
    """
    Save product data to a 'dati_prodotti.txt' file in the format 'name,quantity,purchase_price,selling_price'.
    """
    with open('dati_prodotti.txt', 'w') as file:
        for product in products:
            file.write(f"{product['nome']},{product['quantita']},{product['prezzo_acquisto']},{product['prezzo_vendita']}\n")


In [4]:
def add_product(products):
    """
    Add a new product to the product list or update the quantity of an existing product.
    Requests the user to input the name, quantity, purchase price, and selling price of the new product.
    """
    try:
        name = input("Nome del prodotto: ")

        while True:
            try:
                quantity = int(input("Quantità: "))
                if quantity <= 0:
                    print("La quantità deve essere maggiore di zero.")
                else:
                    break
            except ValueError:
                print("Inserire un valore intero valido per la quantità.")

        purchase_price = float(input("Prezzo di acquisto: "))
        selling_price = float(input("Prezzo di vendita: "))

        if purchase_price <= 0 or selling_price <= 0:
            raise ValueError("Il prezzo di acquisto e il prezzo di vendita devono essere maggiori di zero.")

        present = False
        for product in products:
            if product['nome'] == name:
                product['quantita'] += quantity
                product['prezzo_acquisto'] = purchase_price
                product['prezzo_vendita'] = selling_price
                present = True
                print(f"Aggiornato: {quantity} X {name}")
                break

        if not present:
            products.append({
                'nome': name,
                'quantita': quantity,
                'prezzo_acquisto': purchase_price,
                'prezzo_vendita': selling_price
            })
            print(f"Aggiunto: {quantity} X {name}")

    except ValueError as ve:
        print(f"Errore: {ve}")


In [5]:
def list_products(products):
    """
    Print to the screen the list of available products with their respective name, quantity, and selling price.
    """
    print("PRODOTTO\tQUANTITA'\tPREZZO DI VENDITA")
    products_with_quantity = [product for product in products if product['quantita'] > 0]

    for product in products_with_quantity:
        print(f"{product['nome']}\t{product['quantita']}\t\t€{product['prezzo_vendita']:.2f}")

In [6]:
def register_sale(products):
    """
    Record a sale of one or more products by subtracting the sold quantities from the stock availability.
    """
    sales_recorded = []

    while True:
        product_name = input("Nome del prodotto venduto: ")
        try:
            present = False
            for product in products:
                if product['nome'] == product_name:
                    present = True
                    quantity_sold = int(input(f"Quantità di {product_name} venduta: "))

                    if product['quantita'] >= quantity_sold:
                        sales_recorded.append({
                            'nome': product_name,
                            'quantita': quantity_sold,
                            'prezzo_unitario': product['prezzo_vendita']
                        })
                        product['quantita'] -= quantity_sold
                        print(f"- {quantity_sold} X {product_name}: €{product['prezzo_vendita'] * quantity_sold:.2f}")

                        
                        update_data_file(product)
                    else:
                        print("Quantità non disponibile nel magazzino.")
                    break

            if not present:
                print(f"Il prodotto {product_name} non è presente nel magazzino.")

        except ValueError:
            print("Inserire una quantità valida.")

        add_another = input("Aggiungere un altro prodotto? (si/no): ")
        if add_another.lower() == 'no':
            break

    total_sales = sum(sale['quantita'] * sale['prezzo_unitario'] for sale in sales_recorded)
    print(f"Totale: €{total_sales:.2f}")


In [7]:
def update_data_file(product):
    """
    Update the 'dati_prodotti.txt' file with the new product details after a sale.
    """
    try:
        with open('dati_prodotti.txt', 'r') as file:
            lines = file.readlines()

        with open('dati_prodotti.txt', 'w') as file:
            for line in lines:
                product_data = line.strip().split(',')
                if product_data[0] == product['nome']:
                    new_line = ','.join([
                        product['nome'],
                        str(product['quantita']),
                        str(product['prezzo_acquisto']),
                        str(product['prezzo_vendita'])
                    ]) + '\n'
                    file.write(new_line)
                else:
                    file.write(line)
    except FileNotFoundError:
        print("File non trovato.")

In [8]:
def calculate_profits(products):
    """
    Calculate and print the gross and net profits to the screen.
    Gross profit is the total sales, while net profit is calculated as gross profit minus the cost of product purchases.
    """
    if not products or all(product['quantita'] == 0 for product in products): 
        print("Profitto: lordo=€0.00 netto=€0.00")
        return

    total_purchase_cost = sum(product['prezzo_acquisto'] * product['quantita'] for product in products if product['quantita'] > 0)
    total_revenue = sum(product['prezzo_vendita'] * product['quantita'] for product in products if product['quantita'] > 0)
    
    gross_profit = total_revenue
    net_profit = gross_profit - total_purchase_cost

    print(f"Profitto: lordo=€{gross_profit:.2f} netto=€{net_profit:.2f}")

In [9]:
def show_help_menu():
    """
    Display the list of available commands for the user.
    """
    print("I comandi disponibili sono i seguenti:")
    print("aggiungi: aggiungi un prodotto al magazzino")
    print("elenca: elenca i prodotto in magazzino")
    print("vendita: registra una vendita effettuata")
    print("profitti: mostra i profitti totali")
    print("aiuto: mostra i possibili comandi")
    print("chiudi: esci dal programma")

In [10]:
def main():
    """
    Main function that manages user interaction and the program flow.
    """
    commands_available = [
        'aggiungi',
        'elenca',
        'vendita',
        'profitti',
        'aiuto',
        'chiudi'
    ]

    products = load_data()

    while True:
        command = input("Inserisci un comando (Scrivi 'aiuto' per vedere i comandi elencati): ")

        if command in commands_available:
            if command == 'aggiungi':
                add_product(products)
            elif command == 'elenca':
                list_products(products)
            elif command == 'vendita':
                register_sale(products)
            elif command == 'profitti':
                calculate_profits(products)
            elif command == 'aiuto':
                show_help_menu()
            elif command == 'chiudi':
                save_data(products)
                print("Arrivederci!")
                break
        else:
            print("Comando non valido")
            show_help_menu()


if __name__ == "__main__":
    main()


Inserisci un comando (Scrivi 'aiuto' per vedere i comandi elencati): elenca
PRODOTTO	QUANTITA'	PREZZO DI VENDITA
tofu	3		€8.00
Inserisci un comando (Scrivi 'aiuto' per vedere i comandi elencati): vendita
Nome del prodotto venduto: tofu
Quantità di tofu venduta: 1
- 1 X tofu: €8.00
Aggiungere un altro prodotto? (si/no): chiudi
Nome del prodotto venduto: no
Il prodotto no non è presente nel magazzino.
Aggiungere un altro prodotto? (si/no): no
Totale: €8.00
Inserisci un comando (Scrivi 'aiuto' per vedere i comandi elencati): chiudi
Arrivederci!
